In [1]:
#95%
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import helpermethods
import tensorflow as tf
from tensorflow.python.keras.backend import set_session
config = tf.compat.v1.ConfigProto() 
config.gpu_options.allow_growth = True  
config.log_device_placement = True  
sess2 = tf.compat.v1.Session(config=config)
set_session(sess2)
import numpy as np
import sys
from tensorflow.keras.layers import Dense, Input, RNN
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MSE


#FastRNN and FastGRNN imports
from rnn import FastGRNNCellKeras, FastRNNCellKeras
from data_utils import *

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:21:00.0, compute capability: 8.6



In [2]:
f = '/home/nesl/209as_sec/human_act/Data/Activity_Dataset/'
model_dir = 'trained_models/'
window_size = 550
stride = 50

X_tr, Y_tr, X_test, Y_test = import_auritus_activity_dataset(dataset_folder = f, 
                                use_timestamp=False, 
                                shuffle=True, 
                                window_size = window_size, stride = stride, 
                                return_test_set = True, test_set_size = 300)
print(X_tr.shape)
print(Y_tr.shape)
print(X_test.shape)
print(Y_test.shape)

Xtrain = X_tr
Ytrain = Y_tr
Xtest = X_test
Ytest = Y_test
numClasses = Y_tr.shape[1]
dataDimension = Xtrain[0].shape
channels = Xtrain.shape[2]

100%|█████████████████████████████████████████| 399/399 [00:11<00:00, 34.28it/s]


(14671, 550, 6)
(14671, 9)
(300, 550, 6)
(300, 9)


In [3]:
cell = "FastRNN" # Choose between FastGRNN, FastRNN  ###, UGRNN, GRU and LSTM

inputDims = [window_size, channels] #features taken in by RNN in one timestep
hiddenDims = 32 #hidden state of RNN

totalEpochs = 300
batchSize = 100

learningRate = 0.01
decayStep = 200
decayRate = 0.1

outFile = None #provide your file, if you need all the logging info in a file

#low-rank parameterisation for weight matrices. None => Full Rank
wRank = None 
uRank = None 

#Sparsity of the weight matrices. x => 100*x % are non-zeros
#Note: Sparsity inducing is not supported in this code. 
sW = 1.0 
sU = 1.0

#Non-linearities for the RNN architecture. Can choose from "tanh, sigmoid, relu, quantTanh, quantSigm"
update_non_linearity = "tanh"
gate_non_linearity = "sigmoid"


FastCell = FastRNNCellKeras(hiddenDims, update_non_linearity=update_non_linearity,
                           wRank=wRank, uRank=uRank)

In [4]:
x = inputs = Input(shape=inputDims, name='input')
layer_shape = K.int_shape(x)#x.get_shape()
print(layer_shape)
x = RNN(FastCell, return_sequences=False, name='rnn')(x)
out = Dense(numClasses, activation='softmax', name='dense')(x)
model = Model(inputs=inputs, outputs=out)
Adam_ = Adam()
model.compile(optimizer=Adam_, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

(None, 550, 6)
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 550, 6)]          0         
_________________________________________________________________
rnn (RNN)                    (None, 32)                1250      
_________________________________________________________________
dense (Dense)                (None, 9)                 297       
Total params: 1,547
Trainable params: 1,547
Non-trainable params: 0
_________________________________________________________________


/home/nesl/swapnil_debug/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer.py:1331: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '


In [ ]:
model_name = 'fastrnn.h5'
checkpoint = ModelCheckpoint(model_name, save_best_only=True, monitor='val_accuracy')
model.fit(Xtrain, Ytrain, batchSize, epochs=totalEpochs, validation_data=(Xtest, Ytest),callbacks=[checkpoint])

In [ ]:
model = load_model('fastrnn.h5', custom_objects={'FastRNNCellKeras':FastCell})
model.evaluate(Xtest,Ytest)